In [2]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

ERROR:absl:module 'tensorflow_datasets.core' has no attribute 'utils'
Traceback (most recent call last):
  File "d:\software\Anaconda3\lib\site-packages\tensorflow_datasets\__init__.py", line 59, in <module>
    from tensorflow_datasets import audio
  File "d:\software\Anaconda3\lib\site-packages\tensorflow_datasets\audio\__init__.py", line 19, in <module>
    from tensorflow_datasets.audio.commonvoice import CommonVoice
  File "d:\software\Anaconda3\lib\site-packages\tensorflow_datasets\audio\commonvoice.py", line 27, in <module>
    import tensorflow_datasets.public_api as tfds
  File "d:\software\Anaconda3\lib\site-packages\tensorflow_datasets\public_api.py", line 57, in <module>
    deprecated = core.utils.docs.deprecated(deprecated)
AttributeError: module 'tensorflow_datasets.core' has no attribute 'utils'


###  MobileBERT

In [2]:
# 加载模型
mb_spec = model_spec.get('mobilebert_classifier')

In [3]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=False)

In [4]:
# 训练模型
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=3)

1403/1403 [==============================] - 9194s 7s/step - loss: 0.3904 - test_accuracy: 0.8542
Epoch 2/2
1403/1403 [==============================] - 8641s 6s/step - loss: 0.1283 - test_accuracy: 0.9544
Epoch 3/3
1403/1403 [==============================] - 8577s 6s/step - loss: 0.0753 - test_accuracy: 0.9760


In [5]:
# 检查详细的模型结构。
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 hub_keras_layer_v1v2 (HubKeras  (None, 512)         24581888    ['input_word_ids[0][0]',         
 LayerV1V2)                                                       'input_mask[0][0]',         

In [6]:
# 评估模型
loss, acc = model.evaluate(test_data)

28/28 [==============================] - 40s 1s/step - loss: 0.3856 - test_accuracy: 0.9014


In [7]:
# 导出为 TensorFlow Lite 模型
model.export(export_dir='.')

In [ ]:
# 使用 evaluate_tflite 方法对 TFLite 模型进行评估，以衡量其准确率。
accuracy = model.evaluate_tflite('model.tflite', test_data)
print('TFLite model accuracy: ', accuracy)